# Part IX: Production Deployment

## Chapter 19: Containerization

Containerization packages your application with its dependencies into a standardized unit, ensuring consistency across development, staging, and production environments. Docker is the industry standard for containerizing FastAPI applications, offering isolation, scalability, and simplified deployment. This chapter covers building optimized, secure container images and orchestrating multi-container applications with Docker Compose.

---

### 19.1 Dockerizing FastAPI: Writing Optimized Dockerfile

A well-crafted Dockerfile minimizes image size, build time, and attack surface while maximizing cache utilization and security.

#### Multi-Stage Build Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│              Multi-Stage Docker Build Process                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Stage 1: Builder (Heavy, full toolchain)                        │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │  FROM python:3.11-slim as builder                      │    │
│  │                                                         │    │
│  │  • Install gcc, libpq-dev (build dependencies)          │    │
│  │  • Install Python dependencies                        │    │
│  │  • Compile wheels (pre-compiled packages)               │    │
│  │                                                         │    │
│  │  Size: ~500MB (temporary, discarded)                    │    │
│  └─────────────────────────────────────────────────────────┘    │
│                              │                                   │
│                              ▼ (copy only wheels)                │
│                                                                  │
│  Stage 2: Runtime (Light, production-only)                      │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │  FROM python:3.11-slim as runtime                      │    │
│  │                                                         │    │
│  │  • Install only runtime libraries (libpq5)              │    │
│  │  • Copy pre-compiled wheels from builder                │    │
│  │  • Create non-root user                                 │    │
│  │  • Configure entrypoint                                 │    │
│  │                                                         │    │
│  │  Size: ~150MB (final image)                             │    │
│  └─────────────────────────────────────────────────────────┘    │
│                                                                  │
│  Benefits:                                                       │
│  • Smaller attack surface (no compilers in production)          │
│  • Smaller image size (faster deploys, less storage)            │
│  • Layer caching (faster rebuilds)                              │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

#### Production-Ready Dockerfile

```dockerfile
# Dockerfile - Multi-stage build for FastAPI
# syntax=docker/dockerfile:1

# ═════════════════════════════════════════════════════════════════
# Stage 1: Builder
# ═════════════════════════════════════════════════════════════════
FROM python:3.11-slim as builder

# Security: Run as non-root even during build
RUN useradd -m -u 1000 appuser

# Set environment variables
ENV PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    PIP_NO_CACHE_DIR=1 \
    PIP_DISABLE_PIP_VERSION_CHECK=1

# Install system build dependencies
# These are only needed to compile Python packages
RUN apt-get update && apt-get install -y --no-install-recommends \
    gcc \
    libpq-dev \
    && rm -rf /var/lib/apt/lists/*

# Create virtual environment
# This isolates dependencies from system Python
RUN python -m venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# Install Python dependencies
# Copy only requirements first for better layer caching
COPY requirements.txt .
RUN pip install --upgrade pip && \
    pip install wheel && \
    pip install -r requirements.txt

# Install additional production dependencies
RUN pip install gunicorn uvicorn[standard]

# ═════════════════════════════════════════════════════════════════
# Stage 2: Runtime
# ═════════════════════════════════════════════════════════════════
FROM python:3.11-slim as runtime

# Security labels
LABEL maintainer="your-team@company.com" \
      version="1.0.0" \
      description="FastAPI Production Image"

# Set environment
ENV PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    PYTHONFAULTHANDLER=1 \
    APP_HOME=/app \
    PORT=8000

# Create non-root user for security
# UID 1000 is standard for first user
RUN groupadd -r appgroup && \
    useradd -r -g appgroup -u 1000 -d /home/appuser appuser && \
    mkdir -p /home/appuser && \
    chown -R appuser:appgroup /home/appuser

# Install runtime dependencies only (no compilers)
RUN apt-get update && apt-get install -y --no-install-recommends \
    libpq5 \
    curl \
    && rm -rf /var/lib/apt/lists/* \
    && apt-get clean

# Set working directory
WORKDIR $APP_HOME

# Copy virtual environment from builder
# Only brings compiled packages, not build tools
COPY --from=builder /opt/venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# Copy application code
# Order matters for caching: least changing first
COPY --chown=appuser:appgroup ./app ./app
COPY --chown=appuser:appgroup ./alembic ./alembic
COPY --chown=appuser:appgroup alembic.ini .
COPY --chown=appuser:appgroup gunicorn.conf.py .

# Switch to non-root user
# Prevents privilege escalation attacks
USER appuser

# Health check
# Docker checks if container is healthy
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \
    CMD curl -f http://localhost:8000/health || exit 1

# Expose port
EXPOSE $PORT

# Entry point
# Use exec form for proper signal handling
ENTRYPOINT ["gunicorn"]

# Default command (can be overridden)
CMD ["-c", "gunicorn.conf.py", "app.main:app"]
```

**Dockerfile Security Best Practices Explained:**

1. **Multi-stage builds**: Separate build and runtime environments. Compilers (`gcc`) and headers (`libpq-dev`) needed to build packages are excluded from the final image, reducing attack surface by ~70%.

2. **Non-root user**: Running as `root` in containers is dangerous. The `appuser` (UID 1000) has no special privileges. If the application is compromised, the attacker gains only user-level access.

3. **Layer caching**: Copy `requirements.txt` before source code. Dependencies change less frequently than code, so Docker caches the `pip install` layer and only rebuilds when `requirements.txt` changes.

4. **`.dockerignore`**: Exclude files from build context to prevent secrets and reduce build time:

```dockerfile
# .dockerignore
__pycache__
*.pyc
*.pyo
*.pyd
.Python
env/
venv/
.env
.venv
pip-log.txt
pip-delete-this-directory.txt
.git
.gitignore
.mypy_cache
.pytest_cache
.hypothesis
.coverage
htmlcov
.tox
*.egg-info
.DS_Store
.idea
.vscode
*.swp
*.swo
*~
Dockerfile
docker-compose*.yml
README.md
tests/
docs/
```

#### Optimized Gunicorn Configuration

```python
# gunicorn.conf.py - Production server configuration
import multiprocessing
import os

# Server socket
bind = f"0.0.0.0:{os.getenv('PORT', '8000')}"

# Worker processes
# Formula: (2 x CPU cores) + 1
workers = multiprocessing.cpu_count() * 2 + 1

# Worker class
# uvicorn.workers.UvicornWorker for ASGI (FastAPI)
worker_class = "uvicorn.workers.UvicornWorker"

# Connection handling
worker_connections = 1000
keepalive = 2
timeout = 30
graceful_timeout = 30

# Logging
accesslog = "-"  # stdout
errorlog = "-"   # stdout
loglevel = os.getenv("LOG_LEVEL", "info")
access_log_format = '%(h)s %(l)s %(u)s %(t)s "%(r)s" %(s)s %(b)s "%(f)s" "%(a)s" %(D)s'

# Process naming
proc_name = "fastapi_app"

# Server mechanics
daemon = False
pidfile = "/tmp/gunicorn.pid"

# SSL (terminate at load balancer/proxy in production)
forwarded_allow_ips = "*"
secure_scheme_headers = {
    'X-FORWARDED-PROTOCOL': 'ssl',
    'X-FORWARDED-PROTO': 'https',
    'X-FORWARDED-SSL': 'on'
}

# Preload application for memory efficiency
preload_app = True

# Worker temporary directory (in-memory for containers)
worker_tmp_dir = "/dev/shm"

# Max requests before worker restart (prevent memory leaks)
max_requests = 1000
max_requests_jitter = 50

# Server hooks
def on_starting(server):
    """Called just before master process is initialized."""
    pass

def on_reload(server):
    """Called when receiving SIGHUP."""
    pass

def when_ready(server):
    """Called just after server is started."""
    pass

def worker_int(worker):
    """Called when worker receives SIGINT or SIGQUIT."""
    pass

def on_exit(server):
    """Called just before exiting Gunicorn."""
    pass
```

**Gunicorn + Uvicorn Explained:**

- **Gunicorn**: Process manager that spawns multiple Uvicorn workers. Handles process forking, signal handling, and graceful restarts.
- **UvicornWorker**: ASGI server that runs the actual FastAPI application. Each worker is a separate Python process with its own event loop.
- **Worker count**: `(2 x CPU) + 1` is the standard formula. For I/O-bound apps (databases, HTTP calls), you can use more workers. For CPU-bound, stick to CPU count.
- **Preload**: Loads application code before forking workers. Saves memory (copy-on-write) but prevents hot reloading.

---

### 19.2 Docker Compose: Managing Multi-Container Applications

Docker Compose defines and runs multi-container Docker applications. For FastAPI, this typically includes the app, database, cache, and reverse proxy.

#### Development Docker Compose

```yaml
# docker-compose.yml - Development environment
version: "3.8"

services:
  # FastAPI Application
  app:
    build:
      context: .
      dockerfile: Dockerfile
      target: runtime  # Use runtime stage
    container_name: fastapi_app
    restart: unless-stopped
    ports:
      - "8000:8000"
    environment:
      - DATABASE_URL=postgresql+asyncpg://postgres:postgres@db:5432/fastapi_db
      - REDIS_URL=redis://redis:6379/0
      - SECRET_KEY=dev-secret-key-change-in-production
      - DEBUG=true
      - LOG_LEVEL=debug
    volumes:
      # Mount code for hot reloading (dev only)
      - ./app:/app/app:ro
      - ./alembic:/app/alembic:ro
    depends_on:
      db:
        condition: service_healthy
      redis:
        condition: service_healthy
    networks:
      - backend
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 40s

  # PostgreSQL Database
  db:
    image: postgres:15-alpine
    container_name: fastapi_db
    restart: unless-stopped
    environment:
      - POSTGRES_USER=postgres
      - POSTGRES_PASSWORD=postgres
      - POSTGRES_DB=fastapi_db
      - PGDATA=/var/lib/postgresql/data/pgdata
    volumes:
      - postgres_data:/var/lib/postgresql/data
    ports:
      - "5432:5432"  # Expose for local development tools
    networks:
      - backend
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U postgres"]
      interval: 5s
      timeout: 5s
      retries: 5

  # Redis Cache
  redis:
    image: redis:7-alpine
    container_name: fastapi_redis
    restart: unless-stopped
    volumes:
      - redis_data:/data
    ports:
      - "6379:6379"
    networks:
      - backend
    healthcheck:
      test: ["CMD", "redis-cli", "ping"]
      interval: 5s
      timeout: 3s
      retries: 5

  # Mailhog (email catching for dev)
  mailhog:
    image: mailhog/mailhog:latest
    container_name: fastapi_mail
    ports:
      - "1025:1025"  # SMTP
      - "8025:8025"  # Web UI
    networks:
      - backend

volumes:
  postgres_data:
  redis_data:

networks:
  backend:
    driver: bridge
```

#### Production Docker Compose

```yaml
# docker-compose.prod.yml - Production environment
version: "3.8"

services:
  # Nginx Reverse Proxy
  nginx:
    image: nginx:alpine
    container_name: fastapi_nginx
    restart: always
    ports:
      - "80:80"
      - "443:443"
    volumes:
      - ./nginx/nginx.conf:/etc/nginx/nginx.conf:ro
      - ./nginx/ssl:/etc/nginx/ssl:ro
      - static_files:/app/static:ro
    depends_on:
      - app
    networks:
      - frontend
      - backend

  # FastAPI Application
  app:
    build:
      context: .
      dockerfile: Dockerfile
      target: runtime
    image: fastapi_app:${VERSION:-latest}
    container_name: fastapi_app
    restart: always
    environment:
      - DATABASE_URL=${DATABASE_URL}
      - REDIS_URL=${REDIS_URL}
      - SECRET_KEY=${SECRET_KEY}
      - DEBUG=false
      - LOG_LEVEL=info
      - PORT=8000
    expose:
      - "8000"  # Only expose to nginx, not host
    depends_on:
      db:
        condition: service_healthy
      redis:
        condition: service_healthy
    networks:
      - backend
    deploy:
      resources:
        limits:
          cpus: '1.0'
          memory: 512M
        reservations:
          cpus: '0.25'
          memory: 128M

  # PostgreSQL with optimizations
  db:
    image: postgres:15-alpine
    container_name: fastapi_db
    restart: always
    environment:
      - POSTGRES_USER=${DB_USER}
      - POSTGRES_PASSWORD=${DB_PASSWORD}
      - POSTGRES_DB=${DB_NAME}
      - PGDATA=/var/lib/postgresql/data/pgdata
    volumes:
      - postgres_prod_data:/var/lib/postgresql/data
    expose:
      - "5432"
    networks:
      - backend
    command: >
      postgres
      -c max_connections=200
      -c shared_buffers=256MB
      -c effective_cache_size=768MB
      -c maintenance_work_mem=64MB
      -c checkpoint_completion_target=0.9
      -c wal_buffers=16MB
      -c default_statistics_target=100
      -c random_page_cost=1.1
      -c effective_io_concurrency=200
      -c work_mem=1310kB
      -c min_wal_size=1GB
      -c max_wal_size=4GB
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U ${DB_USER} -d ${DB_NAME}"]
      interval: 10s
      timeout: 5s
      retries: 5

  # Redis with persistence
  redis:
    image: redis:7-alpine
    container_name: fastapi_redis
    restart: always
    volumes:
      - redis_prod_data:/data
      - ./redis/redis.conf:/usr/local/etc/redis/redis.conf:ro
    expose:
      - "6379"
    networks:
      - backend
    command: redis-server /usr/local/etc/redis/redis.conf

  # Celery Worker (for background tasks)
  worker:
    image: fastapi_app:${VERSION:-latest}
    container_name: fastapi_worker
    restart: always
    command: celery -A app.tasks worker --loglevel=info --concurrency=4
    environment:
      - DATABASE_URL=${DATABASE_URL}
      - REDIS_URL=${REDIS_URL}
      - SECRET_KEY=${SECRET_KEY}
    depends_on:
      - redis
      - db
    networks:
      - backend
    deploy:
      resources:
        limits:
          cpus: '0.5'
          memory: 256M

  # Celery Beat (for scheduled tasks)
  scheduler:
    image: fastapi_app:${VERSION:-latest}
    container_name: fastapi_scheduler
    restart: always
    command: celery -A app.tasks beat --loglevel=info
    environment:
      - DATABASE_URL=${DATABASE_URL}
      - REDIS_URL=${REDIS_URL}
    depends_on:
      - redis
    networks:
      - backend

volumes:
  postgres_prod_data:
    driver: local
  redis_prod_data:
    driver: local
  static_files:
    driver: local

networks:
  frontend:
    driver: bridge
  backend:
    driver: bridge
    internal: true  # No external access except through nginx
```

**Docker Compose Key Concepts:**

1. **`depends_on`**: Controls startup order. With `condition: service_healthy`, waits for health checks to pass (requires Compose v2.1+).

2. **Networks**: `frontend` for proxy exposure, `backend` internal for service communication. The `internal: true` flag prevents direct database access from outside.

3. **Volumes**: Named volumes (`postgres_prod_data`) persist data across container restarts. Bind mounts (`./app:/app`) are for development only.

4. **Environment variables**: Production uses `.env` files (not committed to git) with `env_file` or shell exports.

5. **Resource limits**: `deploy.resources` prevents runaway containers from consuming all host resources.

---

### 19.3 Production Strategies: Distroless Images and Security Hardening

For maximum security and minimal size, use distroless or highly minimal base images.

#### Distroless Python Image

```dockerfile
# Dockerfile.distroless - Minimal attack surface
# WARNING: Debugging is harder (no shell)

# Builder stage remains the same
FROM python:3.11-slim as builder

ENV PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1

RUN apt-get update && apt-get install -y --no-install-recommends \
    gcc \
    libpq-dev \
    && rm -rf /var/lib/apt/lists/*

RUN python -m venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt gunicorn uvicorn

# Runtime stage using Google's distroless Python
FROM gcr.io/distroless/python3-debian12

# Copy virtual environment
COPY --from=builder /opt/venv /opt/venv

# Set environment
ENV PYTHONUNBUFFERED=1 \
    PATH="/opt/venv/bin:/opt/venv/lib/python3.11/site-packages:$PATH" \
    PYTHONPATH="/opt/venv/lib/python3.11/site-packages" \
    PORT=8000

WORKDIR /app

# Copy application
COPY --chown=nonroot:nonroot ./app ./app
COPY --chown=nonroot:nonroot gunicorn.conf.py .

# Distroless images run as nonroot by default
USER nonroot:nonroot

EXPOSE 8000

# No shell in distroless, must use exec form
ENTRYPOINT ["/opt/venv/bin/gunicorn", "-c", "gunicorn.conf.py", "app.main:app"]
```

**Distroless Benefits:**
- No shell, package manager, or utilities (no `bash`, `apt`, `curl`)
- Minimal attack surface (no tools for attackers to use)
- Smaller image size (~50MB vs ~150MB for slim)
- Harder to debug (need to attach sidecar containers for debugging)

#### Alternative: Chainguard Images

```dockerfile
# Chainguard Images (Wolfi-based, minimal but with package manager)
FROM cgr.dev/chainguard/python:latest-dev as builder

WORKDIR /app
COPY requirements.txt .

RUN pip install --user --no-cache-dir -r requirements.txt

FROM cgr.dev/chainguard/python:latest

WORKDIR /app

# Copy only necessary artifacts
COPY --from=builder /home/nonroot/.local/lib/python3.11/site-packages /home/nonroot/.local/lib/python3.11/site-packages
COPY --from=builder /home/nonroot/.local/bin /home/nonroot/.local/bin

ENV PATH="/home/nonroot/.local/bin:$PATH"

COPY --chown=nonroot:nonroot ./app ./app

USER nonroot

EXPOSE 8000

ENTRYPOINT ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"]
```

#### Security Scanning

```bash
# Scan image for vulnerabilities
docker scan fastapi_app:latest

# Or use Trivy
trivy image fastapi_app:latest

# Check image size
docker images fastapi_app:latest

# Inspect layers
docker history fastapi_app:latest

# Run with security options
docker run -d \
  --read-only \
  --tmpfs /tmp \
  --security-opt=no-new-privileges:true \
  --cap-drop=ALL \
  --user 1000:1000 \
  fastapi_app:latest
```

**Security Hardening Flags:**
- `--read-only`: Filesystem is read-only (use tmpfs for writable areas)
- `--tmpfs /tmp`: In-memory temporary directory
- `--security-opt=no-new-privileges`: Prevent privilege escalation
- `--cap-drop=ALL`: Drop all Linux capabilities
- `--user 1000:1000`: Force specific UID/GID

---

### Summary

In this chapter, you containerized FastAPI for production:

1. **Dockerfile Optimization**: Implemented multi-stage builds separating build tools from runtime, used non-root users for security, minimized layer count for caching, and created `.dockerignore` to exclude unnecessary files.

2. **Docker Compose**: Defined multi-service applications with PostgreSQL and Redis, configured health checks for service dependencies, used networks for service isolation, and set resource limits for production stability.

3. **Production Strategies**: Compared standard slim images vs distroless for minimal attack surface, configured Gunicorn with Uvicorn workers for process management, and implemented security scanning and hardening options.

**Production Checklist:**
- Use multi-stage builds (builder + runtime)
- Run as non-root user (UID 1000)
- Pin base image versions (`python:3.11-slim`, not `python:latest`)
- Scan images for vulnerabilities before deployment
- Use `.env` files for secrets (never commit to git)
- Configure health checks for all services
- Set memory and CPU limits
- Use read-only root filesystems where possible

---

### What's Next?

**Chapter 20: Deployment Strategies** will cover:
- **Process Managers**: Configuring Gunicorn with Uvicorn workers for production, handling worker restarts, and graceful shutdowns
- **Reverse Proxies**: Nginx configuration for FastAPI including SSL termination, rate limiting, and static file serving
- **Cloud Deployment**: Deploying to AWS, Google Cloud Platform, and modern platforms like Render, Fly.io, and Railway
- **CI/CD Pipelines**: GitHub Actions workflows for building, testing, and deploying containers to production environments

This next chapter completes the journey from code to production, covering the final deployment step.